In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, Resizing, Rescaling
from sklearn.cluster import KMeans

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil

from sklearn.utils import class_weight

In [ ]:
# Extract data
X = np.load('Xtrain_Classification_Part2.npy')
y = np.load('Ytrain_Classification_Part2.npy')

bacc_aug_data = 0
bacc_clusters_a_data = 0
bacc_clusters = 0
bacc_weights_a_aug_data = 0
bacc_weights_a_partially_aug_data_african = 0
bacc_weights_a_partially_aug_data = 0
bacc_weights = 0
bacc_cnn = 0

In [ ]:
for i in range(100):

    # Split in train and validate 
    X_train, X_validate, train_labels, test_labels = train_test_split(X,y)


    # Reshape to 50x50 pixel images
    pixels = 50
    train_images = []
    test_images = []

    for image in X_train:
        train_images.append(image.reshape(pixels, pixels))

    for image in X_validate:
        test_images.append(image.reshape(pixels, pixels))

    train_images = np.array(train_images)
    test_images = np.array(test_images)


    # Normalizing
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    X_train = X_train / 255.0
    X_validate = X_validate / 255.0


    # Sort data based on ethnicity
    caucasian_count = 0
    african_count = 0
    asian_count = 0
    indian_count = 0

    caucasian_train = []
    african_train = []
    asian_train = []
    indian_train = []

    for index, label in enumerate(train_labels):
        if label == 0:
            caucasian_count += 1
            caucasian_train.append(train_images[index])
        elif label == 1:
            african_count += 1
            african_train.append(train_images[index])
        elif label == 2:
            asian_count += 1
            asian_train.append(train_images[index])
        elif label == 3:
            indian_count += 1
            indian_train.append(train_images[index])
        

    # Augment caucasian data by adding mirrored images
    caucasian_train = np.array(caucasian_train)
    caucasian_augmented = []

    caucasian_flipped = caucasian_train[:,:,::-1]

    for index, image in enumerate(caucasian_flipped):
        caucasian_augmented.append(caucasian_train[index])
        caucasian_augmented.append(image)

    caucasian_augmented = np.array(caucasian_augmented)


    # Augment african data by adding mirrored images
    african_train = np.array(african_train)
    african_augmented = []

    african_flipped = african_train[:,:,::-1]

    for index, image in enumerate(african_flipped):
        african_augmented.append(african_train[index])
        african_augmented.append(image)

    african_augmented = np.array(african_augmented)


    # Augment asian data by adding mirrored images
    asian_train = np.array(asian_train)
    asian_augmented = []

    asian_flipped = asian_train[:,:,::-1]

    for index, image in enumerate(asian_flipped):
        asian_augmented.append(asian_train[index])
        asian_augmented.append(image)

    asian_augmented = np.array(asian_augmented)


    # Augment indian data by adding mirrored images
    indian_train = np.array(indian_train)
    indian_augmented = []

    indian_flipped = indian_train[:,:,::-1]

    for index, image in enumerate(indian_flipped):
        indian_augmented.append(indian_train[index])
        indian_augmented.append(image)

    indian_augmented = np.array(indian_augmented)


    # Create clusters
    k = max(african_count, max(asian_count, indian_count))
    clusters = KMeans(n_clusters = k, max_iter = 100, n_init = 5, random_state = 20)
    caucasian_train = np.array(caucasian_train)
    reshaped_data = caucasian_train.reshape(len(caucasian_train),-1)
    clusters_caucasian = clusters.fit_predict(reshaped_data)
    clusters_caucasian = clusters.cluster_centers_

    caucasian_cluster_array = []
    for image in clusters_caucasian:
        caucasian_cluster_array.append(image.reshape(pixels, pixels))
    caucasian_cluster_array = np.array(caucasian_cluster_array)


    # Create new training data
    train_images_aug_data = []
    train_labels_aug_data = []
    train_images_clusters_a_data = []
    train_labels_clusters_a_data = []
    train_images_clusters = []
    train_labels_clusters = []
    train_images_weights_a_aug_data = []
    train_labels_weights_a_aug_data = []
    train_images_weights_a_partially_aug_data_african = []
    train_labels_weights_a_partially_aug_data_african = []
    train_images_weights_a_partially_aug_data = []
    train_labels_weights_a_partially_aug_data = []

    train_images_weights = train_images
    train_labels_weights = train_labels

    count_caucasian = 0
    count_african = 0
    count_asian = 0
    count_indian = 0

    for index, label in enumerate(train_labels):
        if label == 0:
            # Aug data
            train_images_aug_data.append(caucasian_augmented[count_caucasian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(caucasian_augmented[count_caucasian + caucasian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            if count_caucasian < len(caucasian_cluster_array):
                # Clusters a data
                train_images_clusters_a_data.append(caucasian_cluster_array[caucasian_count].reshape(pixels,pixels))
                train_labels_clusters_a_data.append(label)

                # Clusters
                train_images_clusters.append(caucasian_cluster_array[caucasian_count].reshape(pixels,pixels))
                train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(caucasian_augmented[count_caucasian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(caucasian_augmented[count_caucasian + caucasian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_caucasian += 1

        elif label == 1:
            # Aug data
            train_images_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)
            train_images_clusters_a_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)
            train_images_weights_a_partially_aug_data_african.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_african += 1

        elif label == 2:
            # Aug data
            train_images_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_asian += 1

        elif label == 3:
            # Aug data
            train_images_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_indian += 1

    # Cast to array
    train_images_aug_data = np.array(train_images_aug_data)
    train_labels_aug_data = np.array(train_labels_aug_data)
    train_images_clusters_a_data = np.array(train_images_clusters_a_data)
    train_labels_clusters_a_data = np.array(train_labels_clusters_a_data)
    train_images_clusters = np.array(train_images_clusters)
    train_labels_clusters = np.array(train_labels_clusters)
    train_images_weights_a_aug_data = np.array(train_images_weights_a_aug_data)
    train_labels_weights_a_aug_data = np.array(train_labels_weights_a_aug_data)
    train_images_weights_a_partially_aug_data_african = np.array(train_images_weights_a_partially_aug_data_african)
    train_labels_weights_a_partially_aug_data_african = np.array(train_labels_weights_a_partially_aug_data_african)
    train_images_weights_a_partially_aug_data = np.array(train_images_weights_a_partially_aug_data)
    train_labels_weights_a_partially_aug_data = np.array(train_labels_weights_a_partially_aug_data)



    # Build models and train models

    # CNN ---------------------------------------------------------------------------------------------------------
    model = keras.Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation="softmax"))

    model.compile(  optimizer='adam',
                    loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                    metrics = ['accuracy'])
    
    # Train model
    train_images = np.reshape(train_images, (len(train_images), 50, 50, 1))
    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model.fit(train_images, train_labels, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    bacc_cnn += test_acc

    # CNN w aug data ---------------------------------------------------------------------------------------------------------
    model_w_aug_data = keras.Sequential()
    model_w_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_w_aug_data.add(MaxPooling2D((2, 2)))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_w_aug_data.add(MaxPooling2D((2, 2)))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_w_aug_data.add(Flatten())
    model_w_aug_data.add(Dense(64, activation='relu'))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Dense(4, activation="softmax"))

    model_w_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_aug_data = np.reshape(train_images_aug_data, (len(train_images_aug_data), 50, 50, 1))
    train_images_aug_data = np.asarray(train_images_aug_data)
    train_labels_aug_data = np.asarray(train_labels_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_w_aug_data.fit(train_images_aug_data, train_labels_aug_data, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_w_aug_data.evaluate(test_images, test_labels)
    bacc_aug_data += test_acc


    # CNN w clusters a aug data ---------------------------------------------------------------------------------------------------------
    model_clusters_a_data = keras.Sequential()
    model_clusters_a_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_clusters_a_data.add(MaxPooling2D((2, 2)))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters_a_data.add(MaxPooling2D((2, 2)))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters_a_data.add(Flatten())
    model_clusters_a_data.add(Dense(64, activation='relu'))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Dense(4, activation="softmax"))

    model_clusters_a_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_clusters_a_data = np.reshape(train_images_clusters_a_data, (len(train_images_clusters_a_data), 50, 50, 1))
    train_images_clusters_a_data = np.asarray(train_images_clusters_a_data)
    train_labels_clusters_a_data = np.asarray(train_labels_clusters_a_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_clusters_a_data.fit(train_images_clusters_a_data, train_labels_clusters_a_data, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_clusters_a_data.evaluate(test_images, test_labels)
    bacc_clusters_a_data += test_acc


    # CNN w clusters ---------------------------------------------------------------------------------------------------------
    model_clusters = keras.Sequential()
    model_clusters.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_clusters.add(MaxPooling2D((2, 2)))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters.add(MaxPooling2D((2, 2)))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters.add(Flatten())
    model_clusters.add(Dense(64, activation='relu'))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Dense(4, activation="softmax"))

    model_clusters.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_clusters = np.reshape(train_images_clusters, (len(train_images_clusters), 50, 50, 1))
    train_images_clusters = np.asarray(train_images_clusters)
    train_labels_clusters = np.asarray(train_labels_clusters)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_clusters.fit(train_images_clusters, train_labels_clusters, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_clusters.evaluate(test_images, test_labels)
    bacc_clusters += test_acc


    # CNN w weights a aug data ---------------------------------------------------------------------------------------------------------
    model_weights_a_aug_data = keras.Sequential()
    model_weights_a_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_aug_data.add(Flatten())
    model_weights_a_aug_data.add(Dense(64, activation='relu'))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Dense(4, activation="softmax"))

    model_weights_a_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_aug_data = np.reshape(train_images_weights_a_aug_data, (len(train_images_weights_a_aug_data), 50, 50, 1))
    train_images_weights_a_aug_data = np.asarray(train_images_weights_a_aug_data)
    train_labels_weights_a_aug_data = np.asarray(train_labels_weights_a_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_aug_data), train_labels_weights_a_aug_data)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_aug_data.fit(train_images_weights_a_aug_data, train_labels_weights_a_aug_data, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_weights_a_aug_data.evaluate(test_images, test_labels)
    bacc_weights_a_aug_data += test_acc


    # CNN w weights a partially aug data african ---------------------------------------------------------------------------------------------------------
    model_weights_a_partially_aug_data_african = keras.Sequential()
    model_weights_a_partially_aug_data_african.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_partially_aug_data_african.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data_african.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data_african.add(Flatten())
    model_weights_a_partially_aug_data_african.add(Dense(64, activation='relu'))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Dense(4, activation="softmax"))

    model_weights_a_partially_aug_data_african.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_partially_aug_data_african = np.reshape(train_images_weights_a_partially_aug_data_african, (len(train_images_weights_a_partially_aug_data_african), 50, 50, 1))
    train_images_weights_a_partially_aug_data_african = np.asarray(train_images_weights_a_partially_aug_data_african)
    train_labels_weights_a_partially_aug_data_african = np.asarray(train_labels_weights_a_partially_aug_data_african)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_partially_aug_data_african), train_labels_weights_a_partially_aug_data_african)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_partially_aug_data_african.fit(train_images_weights_a_partially_aug_data_african, train_labels_weights_a_partially_aug_data_african, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_weights_a_partially_aug_data_african.evaluate(test_images, test_labels)
    bacc_weights_a_partially_aug_data_african += test_acc


    # CNN w weights a partially aug data ---------------------------------------------------------------------------------------------------------
    model_weights_a_partially_aug_data = keras.Sequential()
    model_weights_a_partially_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_partially_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data.add(Flatten())
    model_weights_a_partially_aug_data.add(Dense(64, activation='relu'))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Dense(4, activation="softmax"))

    model_weights_a_partially_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_partially_aug_data = np.reshape(train_images_weights_a_partially_aug_data, (len(train_images_weights_a_partially_aug_data), 50, 50, 1))
    train_images_weights_a_partially_aug_data = np.asarray(train_images_weights_a_partially_aug_data)
    train_labels_weights_a_partially_aug_data = np.asarray(train_labels_weights_a_partially_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_partially_aug_data), train_labels_weights_a_partially_aug_data)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_partially_aug_data.fit(train_images_weights_a_partially_aug_data, train_labels_weights_a_partially_aug_data, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_weights_a_partially_aug_data.evaluate(test_images, test_labels)
    bacc_weights_a_partially_aug_data += test_acc


    # CNN w weights ---------------------------------------------------------------------------------------------------------
    model_weights = keras.Sequential()
    model_weights.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights.add(MaxPooling2D((2, 2)))
    model_weights.add(Dropout(0.2))
    model_weights.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights.add(MaxPooling2D((2, 2)))
    model_weights.add(Dropout(0.2))
    model_weights.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights.add(Flatten())
    model_weights.add(Dense(64, activation='relu'))
    model_weights.add(Dropout(0.2))
    model_weights.add(Dense(4, activation="softmax"))

    model_weights.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights = np.reshape(train_images_weights, (len(train_images_weights), 50, 50, 1))
    train_images_weights = np.asarray(train_images_weights)
    train_labels_weights = np.asarray(train_labels_weights)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights), train_labels_weights)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights.fit(train_images_weights, train_labels_weights, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)
    test_loss, test_acc = model_weights.evaluate(test_images, test_labels)
    bacc_weights += test_acc

